# Projet numérique - Calcul différentiel
Antoine Rousseau et David Castro

In [1]:
import autograd
import autograd.numpy as np

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 10] # [width, height] (inches). 

from IPython.display import display

In [2]:
def grad(f):
    g = autograd.grad
    def grad_f(x, y):
        return np.array([g(f, 0)(x, y), g(f, 1)(x, y)])
    return grad_f

In [3]:
def J(f):
    j = autograd.jacobian
    def J_f(x, y):
        return np.array([j(f, 0)(x, y), j(f, 1)(x, y)]).T
    return J_f

In [4]:
def display_contour(f, x, y, levels):
    X, Y = np.meshgrid(x, y)
    Z = f(X, Y)
    fig, ax = plt.subplots()
    contour_set = plt.contour(
        X, Y, Z, colors="grey", linestyles="dashed", 
        levels=levels 
    )
    ax.clabel(contour_set)
    plt.grid(True)
    plt.xlabel("$x_1$") 
    plt.ylabel("$x_2$")
    plt.gca().set_aspect("equal")

# Question 1

La courbe de niveau $c$ de $f$ est bornée car, par hypothèse, il existe $A \in \mathbb R$ tel que :
$\forall x \in \mathbb R^2, \vert \vert x \vert \vert \geqslant A \implies f(x) \geqslant c + 1$.
Elle est donc incluse dans $\mathcal B (0, A)$.

C'est également un fermé car $\{c\}$ est un fermé de $\mathbb R$ donc, par
continuité de $f$, $f^{-1}(\{c\})$ est fermé dans $\mathbb R^2$. 

Comme on est en dimension finie, la courbe de niveau $c$ de $f$ est donc un compact de $\mathbb R^2$.

# Question 2

$ \mathbf u =\frac{1}{\vert \vert \nabla f(x_0)\vert \vert}(\partial_2 f(x_0), -\partial_1 f(x_0))$
est le vecteur normé tangent à la courbe de niveau en $x_0$ vers "la droite" quand on regarde dans la direction de
$\nabla f(x_0)$.

En effet, on sait que le gradient en $x_0$ est normal à la courbe. Or
$R = 
\begin{pmatrix}
    0 & 1 \\ -1 & 0
\end{pmatrix}
$
est la matrice de rotation d'angle $-\frac{\pi}{2}$ et $\mathbf u = R \cdot \nabla f(x_0)$.

Donc en notant le vecteur $X = x - x_0$, on obtient $p(x) = (\mathbf u \vert X)$ : $p(x)$ correspond à la projection 
"vers la droite" de $X$ sur la tangente à la courbe en le point considéré.

In [3]:
# Utile a priori pour la tâche 7 (et la 4 ?)

def u(v):
    return np.array([v[1], -v[0]])

# Question 3

On pose $F: (x, t) \in \mathbb R^2 \times \mathbb R \mapsto (f(x) - c, p(x) - t) \in \mathbb R^2$.

On a bien $F$ continûment dérivable car $f$ l'est et $p$ est polynômiale et $(x_0, 0)$ solution de $F(x) = 0$.

De plus, si $(x, t) \in \mathbb R^2 \times \mathbb R$,
$\partial_x F(x, t) =
\begin{pmatrix}
        \partial_1 f(x) &\partial_2 f(x) \\
        \frac{1}{\vert \vert \nabla f(x_0)\vert \vert} \partial_2 f(x_0) 
        &-\frac{1}{\vert \vert \nabla f(x_0)\vert \vert} \partial_1 f(x_0)
\end{pmatrix}$ 
de déterminant strictement négatif en $(x_0, 0)$ donc non nul sur un voisinage de $(x_0, 0)$ par continuité.

$\partial_x F$ est donc inversible sur un voisinage de $(x_0, 0)$.

Le théorème de la fonction implicite nous donne alors le résultat.

# Question 4

D'après le théorème de la fonction implicite, on a également
$\gamma'(t) = d\gamma (t) = -[\partial_x F(\gamma (t), t)]^{-1} \circ \partial_t F(\gamma (t), t)$.

Or, pour tout $(x, t) \in \mathbb R^2 \times \mathbb R$, $\partial_t F(x, t) = (0, -1)$ donc 
$\gamma' (t) = \Delta(\gamma (t)) \times (\partial_2 f(\gamma (t)), -\partial_1 f(\gamma (t)))$
où $\Delta(x) = \frac{\partial_1 f(x) \partial_1 f(x_0) + \partial_2 f(x) \partial_1 f(x_0)}
{\vert \vert \nabla f(x_0)\vert \vert}$.

Donc, par hypothèse, pour tout $t \in ]-\varepsilon, +\varepsilon[$, $\gamma'(t) \ne 0$.

On a bien $\gamma'(t)$ orthogonal à $\nabla f(\gamma(t))$.

# Question 5

In [5]:
N = 100
eps = 10**(-8)

## Tâche 1

In [6]:
def Newton(F, x0, y0, eps=eps, N=N):
    J_F = J(F)
    x, y = x0, y0
    for i in range(N):
        H = np.linalg.solve(J_F(x, y), -F(x, y))
        x += H[0]
        y += H[1]
        if np.sqrt((x - x0)**2 + (y - y0)**2) <= eps:
            return x, y
        x0, y0 = x, y
    else:
        raise ValueError(f"no convergence in {N} steps.")

## Tâche 2

In [7]:
def F(x1, x2):
    return np.array([3.0 * x1 * x1 - 2.0 * x1 * x2 + 3.0 * x2 * x2 - .8, x2 - x1])

In [8]:
Newton(F, .8, .8)

(0.4472135954999579, 0.4472135954999579)

# Question 6

## Tâche 3

In [9]:
def level_curve(f, x0, y0, delta=0.1, N=1000, eps=eps):
    xp, yp = x0, y0
    res = np.empty((2, N))
    res[:, 0] = x0, y0
    for i in range(1, N):
        def F(x, y):
            return (f(x, y) -  c, np.sqrt((x - xp)**2 + (y - yp)**2) - delta)
        cp = Newton(F, xp, yp, eps=eps)
        res[:, i] = cp
        xp, yp = cp
    return res

# définir c
# on a pas le fait qu'il faut aller à gauche

# Question 7

## Tâche 4

Le principal risque est que la fonction traite plusieurs fois une même portion de courbe si elle a fait
"un tour". Dans ce cas, dans des conditions non pathologiques, réaliser le test seulement entre le premier
et le dernier segment suffit. 

Cependant, le programme pourrait faire non pas un tour mais une boucle si la fonction prise en argument est
auto-intersectant. On pourrait dans ce cas faire le test entre le dernier segments et tous ceux qui le précédent.
Avec un test d'intersection entre deux segements en $O(1)$, cela donnerait une complexité totale quadratique 
par rapport au paramètre $N$ dans le pire cas.

Nous avons choisi de ne pas le faire car, au niveau d'une auto-intersection, l'algorithme a potentiellement deux branches à explorer dont une ne l'a peut-être pas encore été. Il est impossible de prévoir a priori le comportement de l'algorithme mais arrêter le programme à ce moment, c'est se priver quoi qu'il arrive d'une exploration d'une "branche" non encore explorée.

De plus, il s'agit bien d'un cas pathologique car une fonction auto-intersectante n'admet pas de tangente en le point d'intersection et donc pas de dérivée partielle. Pour les fonctions avec lesquelles on travaille, on peut considérer que c'est un cas rare.

Donc il faut tester avec le premier segment seulement car sinon des problèmes peuvent apparaître (courbe inachevée) comme dans le cas de la fonction de Rosenbrock.

In [10]:
def det(A, B, C):
    v1, v2 = B - A, C - A
    return v1[0]*v2[1] - v1[1]*v2[0]

La fonction det renvoie $\det (\vec{AB}, \vec{AC})$.

In [11]:
def in_seg(A, B, C):
    e = 10**(-8) # pour prendre en compte les erreurs d'arrondi liées aux doubles
    v1, v2 = B - A, C - A
    return np.abs(det(A, B, C)) <= e and v1[0]*v2[0] - v1[1]*v2[1] <= 0

La fonction in_seg renvoie true si et seulement si $A \in [B, C]$.

In [12]:
def intersection(A, B, C, D):
    e = 10**(-8)
    alignement = in_seg(A, C, D) or in_seg(B, C, D) or in_seg(C, A, B) or in_seg(D, A, B)
    c1, c2 = det(A, B, D), det(A, B, C)
    d1, d2 = det(D, C, A), det(D, C, B)
    return (c1*c2 < 0 and d1*d2 < 0) or alignement

La fonction intersection renvoie true si et seulement si les segements $[A, B]$ et $[C, D]$ s'intersectent.

La variable alignement anticipe les cas "pathologiques" où les segments s'intersectent en un des quatre points considérés. Pour les autres cas, on utilise l'équivalence suivante:

$[A, B] \cap [C, D] \neq \emptyset \iff [A, B] \cap (C, D) \neq \emptyset$ et $(A, B) \cap [C, D] \neq \emptyset$

En effet, on utilise le fait que si $\theta$ est l'angle orienté entre $u$ et $v$ alors
$\det(u, v) = \vert \vert u \vert \vert \vert \vert v \vert \vert \sin \theta$.

Or, la droite $(C, D)$ intersecte $] A, B [$ (le cas des extrêmités est traité dans alignement) si et seulement si les angles $(\vec{CD}, \vec{CA})$ et $(\vec{CD}, \vec{CB})$ ont des orientations strictement opposés donc si et seulement si $\det(\vec{CD}, \vec{CA}) \det(\vec{CD}, \vec{CB}) < 0$.

In [13]:
def level_curve_finie(f, x0, y0, delta = 0.1, N = 1000, eps = eps) :
    xp, yp = x0, y0
    res = np.empty((2, N))
    res[:, 0] = x0, y0
    for i in range(1, N):
        def F(x, y):
            return (f(x, y) - c, np.sqrt((x - xp)**2 + (y - yp)**2) - delta)
        cp = Newton(F, xp, yp, eps=eps)
        res[:, i] = cp
        xp, yp = cp
        if n > 1 and intersection(res[:, 0], cp):
            return res[:, :i + 1:] # on ne renvoie que les points significatifs
    return res

On prend $n > 1$ car les deux premiers segements partagent dans tous les cas un point en commun et on ne veut pas que le programme s'arrête à la première étape.

## Tâche 5

# Question 8

Il faut :

$\begin{pmatrix} a \\ d \end{pmatrix} = \mathbf P_1$,
$\begin{pmatrix} a + b + c \\ d + e + f \end{pmatrix} = \mathbf P_2$,
$\begin{pmatrix} b \\ e \end{pmatrix} = \alpha \mathbf u_1$ et
$\begin{pmatrix} b + c \\ e + f \end{pmatrix} = \beta \mathbf u_2$
pour $a, b, c, d, e, f, g \in \mathbb R$ et $\alpha, \beta \in \mathbb R_+^*$.



Si une solution existe, $a$ et $d$ sont immédiatement déterminés.
Ce problème admet alors une solution si et seulement si les vecteurs $\mathbf P_2 - \mathbf P_1$
et $\mathbf u_2$ sont strictement postivement liés.

En effet, si ce n'est pas le cas, il est clair que le problème n'admet pas de solution.

Réciproquement, si c'est la cas, on est ramené à la résolution d'un système linéaire de 5 inconnues 
à quatre équations. Ce système admet une solution, on peut même imposer $\alpha = 1$.

## Tâche 6

In [ ]:
def positivement_lies(v1, v2):
    e = 10**(-8) # pour prendre en compte les erreurs d'arrondi liés aux doubles
    d = v1[0]*v2[1] - v1[1]*v2[0]
    return np.abs(d) < e and (v1[0]*v2[0] + v1[1]*v2[1] > 0) 

In [17]:
def gamma(t, P1, P2, u1, u2):
    if positivement_lies(P2 - P1, u2): # si le système admet une solution
        a, d = P1
        b, e = u1
        c, f = P2 - P1 - u1 # en prenant alpha = 1 avec les notations précédentes
        return np.array([a + b*t + c*t*t, d + e*t + f*t*t])
    P1.resize((2, 1)) # si le système n'en admet pas
    P2.resize((2, 1))
    t.resize((1, len(t)))
    return P1 + (P2 - P1).dot(t)

In [18]:
# Test

P1 = np.array([0., 0.])
P2 = np.array([1., 1.])
u1 = np.array([-1., 0])
u2 = np.array([1., 0.])
v2 = np.array([1., 1.])

t = np.linspace(0, 1, 10)
gamma(t, P1, P2, u1, v2)

array([[ 0.        , -0.08641975, -0.12345679, -0.11111111, -0.04938272,
         0.0617284 ,  0.22222222,  0.43209877,  0.69135802,  1.        ],
       [ 0.        ,  0.01234568,  0.04938272,  0.11111111,  0.19753086,
         0.30864198,  0.44444444,  0.60493827,  0.79012346,  1.        ]])

## Tâche 7

On parabolise entre deux points en prenant les dérivées de la fonction étudiée
en $P_1$ et $P_2$ pour $u_1$ et $u_2$. On note $N = oversampling$. On répartit les $N - 1$ points sur la parabole obtenue en prenant $t_n = \frac{n}{N}$ pour $n$ variant entre $1$ et $N - 1$. C'est-à-dire qu'on prend les
$\gamma (t_n)$.

In [18]:
def level_curve_interpole(f, x0, y0, delta = 0.1, N = 1000, eps = eps, overslamping = 1) :
    t = np.arange(1, oversampling - 1)/oversampling
    xp, yp = x0, y0
    res = np.empty((2, N*(overslamping+1)))
    res[:, 0] = x0, y0
    for i in range(1, N):
        def F(x, y):
            return (f(x, y) - c, np.sqrt((x - xp)**2 + (y - yp)**2) - delta)
        cp = Newton(F, xp, yp, eps=eps)
        for j in range(1, overslamping) :
            res[:i + j] = gamma(t, res[:,i*overslamping], cp, grad(f)(res[:,i]), grad(f)(cp)) # c'est comme ça qu'on écrit grad ou alors 
        res[:, i*(overslamping+1)] = cp                                                       # c'est grad_f(x,y) ? Mais ensuite je suis pas  
        xp, yp = cp                                                                           # sûr que ce soit le gradient qui nous 
        if n > 1 and intersection(res[:, 0], cp):
            return res[:, :i + 1:] # on ne renvoie que les points significatifs
    return res

## Tâche 8